In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Streaming write to Kafka")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

spark

In [ ]:
# Rates a spark monotonically increasing id dataframe with a rate of 1/second
df_streaming = spark.readStream.format("rate").load()

In [ ]:
df_streaming.printSchema()

In [ ]:
from pyspark.sql import functions as F

df_streaming = df_streaming.withColumn("value", F.expr("cast(value as string)"))

In [ ]:
df_streaming.writeStream.format("kafka").option(
    "kafka.bootstrap.servers", "kafka:9092"
).option("topic", "rate").outputMode("append").option(
    "checkpointLocation",
    f"/home/jovyan/streaming_checkpoint_dir/{spark.sparkContext.appName.replace(' ', '_')}",
).start().awaitTermination()

In [ ]:
spark.stop()